In [114]:
from io import StringIO
import os
import itertools
import pickle
import time
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from collections import Counter
import re
import camelot
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [115]:
index2 = pd.read_csv('F:\Environmental Baseline Data\Version 4 - Final\Indices\Index 2 - PDFs for Major Projects with ESAs.csv', index_col = 0)

In [117]:
path = 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/'
paths = os.listdir(path)
all_paths = [path + str(x) for x in paths]
len(all_paths)

1729

In [118]:
#open pickle file to extact strings startswith figure or title
#7 min execution time

start = time.time()
file_name = []
page_number = []
table_title = []

exceptions_list = ['...', 'Table of Content', 'TABLE OF CONTENTS'] 

for x in all_paths:
    with open(x, 'rb') as f:
        data = pickle.load(f)
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        for b, p in enumerate(pages):
            pages_text = [x.text for x in p.find_all('p')]
            for y in pages_text:
                if y.startswith(('Table', 'TABLE', 'Figure', 'FIGURE')) and not any(substring in y for substring in exceptions_list):
                    file_name.append(x)
                    table_title.append(y)
                    page_number.append(b + 1)
#                     print(y, b+1)
print('Time: ', time.time() - start)

Time:  369.4204022884369


In [119]:
df_table_names = pd.DataFrame({'file_name' : file_name, 'page_number' : page_number, 'table_title' : table_title})
df_table_names.shape
df_table_names['DataID'] = df_table_names['file_name'].str.replace('\n', ' ').str.replace('F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_', '').str.replace('.pkl', '').astype(int)
df_table_names['table_title'] = df_table_names['table_title'].str.replace('\n', ' ')
df_table_names = df_table_names.drop('file_name', axis = 1)

df_table_names_project = df_table_names.merge(index2, on = 'DataID')
df_table_names_project.loc[df_table_names_project['table_title'].str.contains('FIGURE'), 'Category'] = 'Figure'
df_table_names_project.loc[df_table_names_project['table_title'].str.contains('TABLE'), 'Category'] = 'Table'
df_table_names_project.loc[df_table_names_project['table_title'].str.contains('Figure'), 'Category'] = 'Figure'
df_table_names_project.loc[df_table_names_project['table_title'].str.contains('Table'), 'Category'] = 'Table'
df_table_names_project.head()

(33082, 3)

,page_number,table_title,DataID,file_name,application_name,esa_folder_link,DocumentID,esa_download_link,DataID_pdf,Section,...,Consultant Name,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Location,Category
0,5,FIGURE 1 REGIONAL LOCATION,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Figure
1,6,FIGURE 2 LOCATION OF WATERCOURSE CROSSINGS,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Figure
2,8,TABLE 1,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Table
3,9,TABLE 2,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Table
4,14,TABLE 3,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Table


In [120]:
df_table_names_project['Application title short'].nunique()
df_table_names_project.shape
df_table_names_project['table_title'] = df_table_names_project['table_title'].str.strip()

36

(33082, 25)

In [122]:
save_string = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/Table titles raw data/table_titles.csv'
df_table_names_project.to_csv(save_string, index = False, encoding='utf-8-sig')